In [1]:
import pandas as pd
import random
import transformers
import datasets
import sklearn
import bs4

In [2]:
def reformat_sentiment(x):
    return int(x == 'Positive')

def load_dataset(name):
    # load the dataset
    url = 'https://raw.githubusercontent.com/acmi-lab/counterfactually-augmented-data/master/sentiment/combined/paired/' + name
    dataset = pd.read_csv(url, sep='\t')
    dataset.rename(columns={"Sentiment": "sentiment", "Text": "text", "batch_id": "paired_id"}, inplace=True)
    # reformat 'sentiment' column
    dataset['sentiment'] = dataset['sentiment'].apply(lambda value: reformat_sentiment(value))

    return dataset

def random_shuffle_df(df, seed):
    random.seed(seed)
    df = sklearn.utils.shuffle(df)

    return df

def set_example_counter(idx, found_idsx):
    if idx in found_idsx:
        return 0
    else:
        found_idsx[idx] = 0
        return 1

def randomly_assign_conterfactuals(df, seed=1):
    # prepare the proper Dataframe for the dataset
    df = random_shuffle_df(df, seed)
    paired_ids = df['paired_id'].values
    found_ids = {}
    counterfactual_column = []
    for idx in paired_ids:
        counterfactual_column.append(set_example_counter(idx, found_ids))
    df['is_counterfactual'] = counterfactual_column

    return df

# prepare the dataset with input-counterfactuals instances
def prepare_dataframe_with_counterfacuals(df):

    # group by paired_id
    gb = df.groupby(by=["paired_id"])

    # create new columns "example" and "counterfactual"
    example_column = []
    counter_column = []
    paired_id_column = []
    label_ex = []
    label_counter = []
    for group_id in gb.groups: # group_id == paired_id
        group = gb.get_group(group_id)
        is_counterfactual_column = group['is_counterfactual'].values
        text_column = group['text'].values
        sentiment_column = group['sentiment'].values
        for is_counter, text, label in zip(is_counterfactual_column,
                                           text_column,
                                           sentiment_column):
            if is_counter:
                counter_column.append(text)
                label_counter.append(label)
            else:
                example_column.append(text)
                label_ex.append(label)

        paired_id_column.append(group_id)

    # clean the text from html tags
    example_column = [bs4.BeautifulSoup(el, "lxml").text for el in example_column]
    counter_column = [bs4.BeautifulSoup(el, "lxml").text for el in counter_column]

    # add the new columns to a new dataframe
    d = {'paired_id': paired_id_column,
         'example': example_column,
         'label_ex': label_ex,
         'counterfactual': counter_column,
         'label_counter': label_counter}
    df_with_counterfactuals = pd.DataFrame(data=d)
    df_with_counterfactuals.sort_values(by="paired_id", ascending=True, inplace=True)

    return  df_with_counterfactuals

In [3]:
training_set = load_dataset("train_paired.tsv")
dev_set = load_dataset("dev_paired.tsv")
test_set = load_dataset("test_paired.tsv")
print(f"Datasets are of type {type(test_set)}")
print(f"# of samples in the training set:{len(training_set)}")
print(f"# of samples in the dev set:{len(dev_set)}")
print(f"# of samples in the test set:{len(test_set)}")

Datasets are of type <class 'pandas.core.frame.DataFrame'>
# of samples in the training set:3414
# of samples in the dev set:490
# of samples in the test set:976


In [4]:
# append the 3 datasets
imdb_dataframe = pd.concat([training_set, dev_set, test_set], ignore_index=True)
print(f"# of samples:{len(imdb_dataframe)}")
imdb_dataframe.head(2)

# of samples:4880


,sentiment,text,paired_id
0,0,"Long, boring, blasphemous. Never have I been s...",4
1,1,"Long, fascinating, soulful. Never have I been ...",4


In [5]:
random_seed = 5
df_processed = randomly_assign_conterfactuals(imdb_dataframe.copy(deep=True), random_seed)
df_processed = prepare_dataframe_with_counterfacuals(df_processed)
print(f"# of samples:{len(df_processed)}")
df_processed.head(2)

# of samples:2440


,paired_id,example,label_ex,counterfactual,label_counter
0,4,"Long, fascinating, soulful. Never have I been ...",1,"Long, boring, blasphemous. Never have I been s...",0
1,13,"If you haven't seen this, it's terrible. It is...",0,"If you haven't seen this, it's incredible. It ...",1


In [6]:
# split into train-val-test
random_state = 19
train_prop = 0.8 # of the whole dataset
val_prop = 0.2 # of the whole dataset
df_training = df_processed.sample(frac=train_prop, random_state=random_state)
new_val_prop = len(df_processed) * val_prop / len(df_training)

df_test = df_processed.drop(df_training.index)

df_val = df_training.sample(frac=new_val_prop, random_state=random_state)
df_training = df_training.drop(df_val.index)

print(f"# of samples for training:{len(df_training)}")
print(f"# of samples for validation:{len(df_val)}")
print(f"# of samples for test:{len(df_test)}")

# of samples for training:1464
# of samples for validation:488
# of samples for test:488


<h1>Prepare data for T5 (seq2seq language model)</h1>

In [8]:
# Load "gpt2-medium" objetcs
tokenizer = transformers.GPT2Tokenizer.from_pretrained("gpt2-medium")
lm = transformers.GPT2LMHeadModel.from_pretrained("gpt2-medium")
lm_config_class = transformers.GPT2Config.from_pretrained("gpt2-medium")
lm_config_class

GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.14.1",
  "use_cache": true,
  "vocab_size": 50257
}

In [9]:
tokenizer

PreTrainedTokenizer(name_or_path='gpt2-medium', vocab_size=50257, model_max_len=1024, is_fast=False, padding_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)})

In [10]:
training_set = datasets.Dataset.from_pandas(df_training)
val_set = datasets.Dataset.from_pandas(df_val)
test_set = datasets.Dataset.from_pandas(df_test)

In [17]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
def tokenize_function(examples):
    # return tokenizer(examples["example"], padding="max_length", truncation=True)
    return tokenizer(examples["example"])

apply_batch = True
tokenized_train = training_set.map(tokenize_function, batched=apply_batch)
tokenized_val = val_set.map(tokenize_function, batched=apply_batch)
tokenized_test = test_set.map(tokenize_function, batched=apply_batch)
print("Datasets have been tokenized successfully!")

Assigning [PAD] to the pad_token key of the tokenizer


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Datasets have been tokenized successfully!


In [18]:
no_cuda = True
training_args = transformers.TrainingArguments("test_trainer", per_device_train_batch_size=1, no_cuda=no_cuda)
trainer = transformers.Trainer(model=lm, args=training_args, train_dataset=tokenized_train, eval_dataset=tokenized_val)
training_args

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=test_trainer/runs/Feb24_16-29-07_diego-worksta

In [26]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: paired_id, label_ex, counterfactual, example, label_counter, __index_level_0__.
***** Running training *****
  Num examples = 1464
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 4392


KeyboardInterrupt: 

In [23]:
len(tokenized_val['input_ids'][0])

98

In [28]:
# metric = datasets.load_metric("rouge")
metric = datasets.load_metric("bleu")

Downloading:   0%|          | 0.00/2.49k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

In [29]:
metric

Metric(name: "bleu", features: {'predictions': Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), 'references': Sequence(feature=Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), length=-1, id='references')}, usage: """
Computes BLEU score of translated segments against one or more references.
Args:
    predictions: list of translations to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
Returns:
    'bleu': bleu score,
    'precisions': geometric mean of n-gram precisions,
    'brevity_penalty': brevity penalty,
    'length_ratio': ratio of lengths,
    'translation_length': translation_length,
    'reference_length': reference_length
Examples

In [31]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained("gpt2-medium")

loading configuration file https://huggingface.co/gpt2-medium/resolve/main/config.json from cache at /home/diego/.cache/huggingface/transformers/3a7a4b7235202f93d14a4a5e8200709184c5b25a29d9cfa6b0ede5166adf0768.cf0ec4a33a38dc96108560e01338af4bd3360dd859385d451c35b41987ae73ff
Model config GPT2Config {
  "_name_or_path": "gpt2-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_typ

ValueError: Unrecognized configuration class <class 'transformers.models.gpt2.configuration_gpt2.GPT2Config'> for this kind of AutoModel: AutoModelForSeq2SeqLM.
Model type should be one of BigBirdPegasusConfig, M2M100Config, LEDConfig, BlenderbotSmallConfig, MT5Config, T5Config, PegasusConfig, MarianConfig, MBartConfig, BartConfig, BlenderbotConfig, FSMTConfig, XLMProphetNetConfig, ProphetNetConfig, EncoderDecoderConfig.

In [32]:
transformers.AutoModelForCausalLM.from_pretrained("gpt2-medium")

loading configuration file https://huggingface.co/gpt2-medium/resolve/main/config.json from cache at /home/diego/.cache/huggingface/transformers/3a7a4b7235202f93d14a4a5e8200709184c5b25a29d9cfa6b0ede5166adf0768.cf0ec4a33a38dc96108560e01338af4bd3360dd859385d451c35b41987ae73ff
Model config GPT2Config {
  "_name_or_path": "gpt2-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_special_tokens": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_typ

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
     